In [2]:
import pandas as pd
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import networkx as nx
import os  
import ast #to convert the string into dict
from datetime import datetime as dt
import requests
import dask.dataframe as dd


In [9]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [10]:
df_articles = pd.read_csv('./Articles/newsarticles.csv')

In [4]:
df_articles['body_no_html'] = df_articles['body'].apply(remove_html_tags)

stop_words = set(stopwords.words('french'))
# Remove French stopwords and non-alphabetic tokens
tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
id2word = corpora.Dictionary([tokens])
corpus = [id2word.doc2bow([token]) for token in tokens]

In [11]:
#delete rows without user needs (265)
df_articles.dropna(subset=['userNeeds'],inplace = True)
df_articles.reset_index(drop=True,inplace=True)

In [12]:
#the dictionary is reversed --> now we have principal/secondaire as keys
dict_column = []
for i in range(0,df_articles.shape[0]):
    inv_map = dict(zip(ast.literal_eval(df_articles['userNeeds'][i]).values(), ast.literal_eval(df_articles['userNeeds'][i]).keys()))
    dict_column.append(inv_map)

df_articles['userNeeds_dict'] = dict_column
userNeed = []
for i in range(0,df_articles.shape[0]):
    userNeed.append(df_articles['userNeeds_dict'][i].get('principal'))

df_articles['userNeeds_clean'] = userNeed
df_articles.drop(['userNeeds_dict'],axis = 1,inplace=True)

In [14]:
df_articles.to_csv('Articles_withCleanedUN.csv')

## Topic Association

In [4]:
df_rts_0 = dd.read_csv('C:/Users/laura.finarell/OneDrive - HESSO/Polarizzazione/rts_accesslogs/www.rts.ch_accesslogs_2023-03.0/www.rts.ch_accesslogs.log',sep = ' ', header = None,)
df_rts = df_rts_0.compute()

In [5]:
df_rts = df_rts.drop([0,1,3,5,6,7,8,10,11,12,13,14,16,17,18],axis = 1)
df_rts.columns = ['Timestamp','User_IP','Url','User_Agent']
index_html = df_rts.query('Url.str.contains(".html$", na= False)').index
print('Html number')
print(len(index_html))
df_rts_html = df_rts[df_rts.index.isin(index_html)]
df_rts_html.reset_index(inplace=True, drop=True)

Html number
2833789


In [33]:
df_rts_html.loc[65,'Url']

'info/culture/cinema/13891031-drii-winter-cascadeuses-et-la-ligne-recompenses-aux-prix-du-cinema-suisse.html'

In [23]:
for i in range(df_rts_html.shape[0]):
    if re.search('/1',df_rts_html.loc[i,'Url']) is not None:
        print(i)

0
18
20
26
35
43
45
61
65
70
75
81
87
89
95
98
103
114
115
122
123
125
130
143
147
151
154
157
170
189
199
205
211
217
228
230
235
239
244
249
252
256
260
269
278
286
289
294
298
304
308
311
326
329
334
336
346
357
365
372
381
384
391
404
406
407
408
410
414
417
418
433
435
436
442
450
452
456
459
460
463
467
468
469
471
474
479
481
483
485
491
493
502
511
514
515
517
519
524
525
526
531
533
534
544
554
559
562
567
571
573
575
581
584
589
590
594
596
597
601
617
621
643
644
650
654
657
660
668
670
677
691
742
798
816
867
869
870
875
876
878
883
891
893
897
967
973
1006
1011
1024
1029
1035
1037
1038
1068
1099
1109
1135
1150
1172
1202
1220
1264
1279
1283
1287
1327
1351
1352
1353
1398
1404
1408
1409
1427
1431
1481
1494
1495
1524
1551
1561
1562
1566
1567
1570
1575
1576
1580
1583
1585
1589
1590
1593
1595
1605
1607
1616
1619
1626
1628
1630
1634
1638
1644
1648
1655
1662
1666
1667
1673
1684
1685
1686
1692
1698
1700
1702
1708
1721
1722
1726
1731
1736
1739
1747
1753
1756
1758
1770
1777
1783
1788

TypeError: expected string or bytes-like object, got 'float'

In [6]:
topics = {'culture': r'culture',
          'economie': r'economie',
          'suisse': r'suisse',
          'monde': r'monde',
          'sciences-tech': r'sciences-tech',
          'sport': r'sport',
          'environnement' : r'environnement'}

def extract_topic_and_id(url):
    if pd.isna(url):
        return 'other', None
    for topic, pattern in topics.items():
        if re.search(pattern, url):
            escenic_id_match = re.search(r'/(\d+)-', url)
            escenic_id = escenic_id_match.group(1) if escenic_id_match else None
            return topic, escenic_id
    return 'other', None  # If no topic matches, return 'other' and None for Escenic ID

In [ ]:
# Apply the function to extract the topic and Escenic ID
df_rts_html[['Topic', 'Escenic_ID']] = df_rts_html['Url'].apply(lambda x: pd.Series(extract_topic_and_id(x)))

In [8]:
topic, escenic_id = extract_topic_and_id(df_rts_html.loc[65,'Url'])

print("Topic:", topic)
print("Escenic ID:", escenic_id)

Topic: culture
Escenic ID: 13891031
